In [1]:
%load_ext autoreload

In [91]:
%autoreload
import os
from sys import path
import re
import pandas as pd
path.append('../..')
import numpy as np
from scipy.stats import ttest_rel
from relation_extraction.data.summarize import *
from relation_extraction.data.converters.converter_i2b2 import relation_dict
output_path = '/scratch/geeticka/i2b2-analyze/'
def res(path): return os.path.join(output_path, path)
original_sentences_path = os.path.join('/crimea/geeticka/data/relation_extraction/i2b2/pre-processed/original/test_original.txt')
entity_blinded_sentences_path = os.path.join('/crimea/geeticka/data/relation_extraction/i2b2/pre-processed/entity_blinding/test_entity_blinding.txt')
relation_full_form_dictionary = {'None': 'None', 'PIP': 'PIP', 'TeCP': 'TeCP',
                                 'TeRP': 'TeRP', 'TrAP': 'TrAP', 'TrCP': 'TrCP',
                                 'TrIP': 'TrIP', 'TrNAP': 'TrNAP', 'TrWP': 'TrWP'}
relation_as_short_list = ['None', 'PIP', 'TeCP', 'TeRP', 'TrAP', 'TrCP', 'TrIP', 'TrNAP', 'TrWP']

In [70]:
confusion_matrix_portion = '<<< 8-WAY EVALUATION >>>'

In [71]:
summary_cm_baseline, summary_cm_conceptblind = print_full_summary('i2b2',
    'result-baseline-i2b2.txt', 'result-concblind-i2b2.txt', 
    'Baseline','Concept-Blind', res, relation_full_form_dictionary, relation_as_short_list, confusion_matrix_portion, 2)


TTest from Baseline to Concept-Blind
Below is the metric comparsion across the two models considering individual relations, excluding 'Other'
Metric: Precision 	 statistic -0.53 	 p_value 0.6118072355651132
Metric: Recall 	 statistic -3.45 	 p_value 0.010701830797093427
Metric: F1 	 statistic -3.93 	 p_value 0.005686085116100631

TTest from Baseline to Concept-Blind
Below is the metric comparsion across the two models considering individual relations, including 'Other'
Metric: Precision 	 statistic -0.58 	 p_value 0.5794757407237194
Metric: Recall 	 statistic -3.25 	 p_value 0.011693295800340134
Metric: F1 	 statistic -3.66 	 p_value 0.006409085463347231

All the macro and micro metrics are calculated, excluding the 'Other' class
Macro - Micro for the Baseline model
Metric: Precision 	 Macro-Micro -19.56
Metric: Recall 	 Macro-Micro -30.54
Metric: F1 	 Macro-Micro -28.21

Macro - Micro for the Concept-Blind model
Metric: Precision 	 Macro-Micro -26.89
Metric: Recall 	 Macro-Micro -32.

In [72]:
summary_cm_baseline

summary,Gold Relation,Confused With(num_examples),Correct Predictions
0,None,PIP(335) TeCP(12) TeRP(235) TrAP(153) TrCP(16)...,9515.0
1,PIP,None(460) TeRP(9) TrAP(5) TrCP(1) TrIP(41),414.0
2,TeCP,None(27) PIP(3) TeRP(61) TrAP(10) TrIP(25),62.0
3,TeRP,None(209) PIP(21) TeCP(26) TrAP(1) TrCP(2) TrI...,1411.0
4,TrAP,None(140) PIP(21) TeCP(4) TeRP(1) TrCP(4) TrIP...,576.0
5,TrCP,None(64) PIP(4) TeRP(6) TrAP(9) TrIP(23) TrNAP(2),35.0
6,TrIP,None(12) PIP(4) TeRP(1) TrAP(23) TrCP(1),10.0
7,TrNAP,None(19) PIP(4) TeRP(1) TrAP(26) TrCP(2) TrIP(7),16.0
8,TrWP,None(19) PIP(2) TeRP(1) TrAP(11) TrCP(4) TrIP(9),0.0


In [74]:
summary_cm_conceptblind

summary,Gold Relation,Confused With(num_examples),Correct Predictions
0,None,PIP(310) TeCP(25) TeRP(320) TrAP(179) TrCP(38)...,9576.0
1,PIP,None(429) TrIP(3),498.0
2,TeCP,None(26) TeRP(59) TrIP(4),99.0
3,TeRP,None(184) TeCP(23) TrIP(3),1526.0
4,TrAP,None(117) TrCP(12) TrIP(15) TrNAP(4),675.0
5,TrCP,None(52) TrAP(28) TrIP(6) TrNAP(1),56.0
6,TrIP,None(16) TrAP(19) TrCP(4) TrNAP(2),10.0
7,TrNAP,None(16) TrAP(33) TrCP(4) TrIP(2),20.0
8,TrWP,None(24) TrAP(9) TrCP(9) TrIP(3) TrNAP(1),0.0


In [83]:
def read_answers_line(line):
    linenum, relation = line.strip().split()
    return linenum, relation

In [94]:
def asstring(list_of_strings):
    return " ".join(list_of_strings)

In [95]:
def read_sentence_and_entities(line):
    line = line.strip().split()
    sentence = line[5:]
    relation = relation_dict[int(line[0])]
    entity1_idx = (int(line[1]), int(line[2]))
    entity2_idx = (int(line[3]), int(line[4]))
    entity1 = sentence[entity1_idx[0] : entity1_idx[1] + 1]
    entity2 = sentence[entity2_idx[0] : entity2_idx[1] + 1]
    
    return relation, asstring(entity1), asstring(entity2), asstring(sentence)

In [135]:
needed_linenum_and_relation = {}

In [138]:
with open(res('answers_for_dev-baseline.txt')) as textfile1, open(res("answers_for_dev-concblind.txt")) as textfile2, \
open(res('answers_for_dev_gold-baseline.txt')) as textfile3: 
    for x, y, z in zip(textfile1, textfile2, textfile3):
        linenum, baseline_relation = read_answers_line(x)
        _, entity_blind_relation = read_answers_line(y)
        _, gold_relation = read_answers_line(z)
        if entity_blind_relation == gold_relation and baseline_relation != gold_relation:
            needed_linenum_and_relation[int(linenum) - 1] = (baseline_relation, entity_blind_relation)

In [139]:
print('We print the baseline first and then the entity blinded version. Goal relation corresponds to entity blinded\n\n')
curr_linenum = 0
with open(original_sentences_path) as original_sentences, open(entity_blinded_sentences_path) as entity_blinded_sentences:
    for x, y in zip(original_sentences, entity_blinded_sentences):
        if curr_linenum in needed_linenums:
            _, e1_b, e2_b, s_b = read_sentence_and_entities(x.strip())
            _, e1_c, e2_c, s_c = read_sentence_and_entities(y.strip())
            r_b, r_c = needed_linenum_and_relation[curr_linenum]
            print('Predicted Relation: \t {0}, {1} \nEntities: \t {2}, {3} \t {4}, {5} \nSentences: \n\t{6} \n\t {7}'.format(
                    r_b, r_c, e1_b, e1_c, e2_b, e2_c, s_b, s_c))
            print('\n')
        curr_linenum += 1

We print the baseline first and then the entity blinded version. Goal relation corresponds to entity blinded


Predicted Relation: 	 TrAP, None 
Entities: 	 saphenous vein graft, TREATMENT 	 severe triple vessel coronary artery disease, PROBLEM 
Sentences: 
	Cardiac catheterization demonstrates severe triple vessel coronary artery disease , RCA 100% occluded , saphenous vein graft to OM1 graft was 100% occluded and a patent LIMA , but severely diseased LAD . 
	 TEST demonstrates PROBLEM , PROBLEM , TREATMENT to TREATMENT was PROBLEM and a patent LIMA , but PROBLEM .


Predicted Relation: 	 TrIP, TrAP 
Entities: 	 OM1 graft, TREATMENT 	 severely diseased LAD, PROBLEM 
Sentences: 
	Cardiac catheterization demonstrates severe triple vessel coronary artery disease , RCA 100% occluded , saphenous vein graft to OM1 graft was 100% occluded and a patent LIMA , but severely diseased LAD . 
	 TEST demonstrates PROBLEM , PROBLEM , TREATMENT to TREATMENT was PROBLEM and a patent LIMA , but PROBLEM

Predicted Relation: 	 None, TeRP 
Entities: 	 lower abdominal pain, PROBLEM 	 brpbr, PROBLEM 
Sentences: 
	71 y/o male with dm2 , cad , chf , uc presents with one week of lower abdominal pain and 2 episodes of brpbr on the day of admission . 
	 71 y/o male with PROBLEM , PROBLEM , PROBLEM , PROBLEM presents with one week of PROBLEM and 2 episodes of PROBLEM on the day of admission .


Predicted Relation: 	 None, TeRP 
Entities: 	 uc, PROBLEM 	 lower abdominal pain, PROBLEM 
Sentences: 
	71 y/o male with dm2 , cad , chf , uc presents with one week of lower abdominal pain and 2 episodes of brpbr on the day of admission . 
	 71 y/o male with PROBLEM , PROBLEM , PROBLEM , PROBLEM presents with one week of PROBLEM and 2 episodes of PROBLEM on the day of admission .


Predicted Relation: 	 None, PIP 
Entities: 	 cad, PROBLEM 	 lower abdominal pain, PROBLEM 
Sentences: 
	71 y/o male with dm2 , cad , chf , uc presents with one week of lower abdominal pain and 2 episodes of brpbr on the day of 

	 TEST on admission were notable for TEST of 131 , TEST 6.2 , TEST 111 , TEST 3.9 , TEST 23 , TEST 27 with PROBLEM and PROBLEM on TEST , TEST 92 and TEST 496 .


Predicted Relation: 	 TeRP, None 
Entities: 	 LDH, TEST 	 2+ schistocytes, PROBLEM 
Sentences: 
	Laboratory studies on admission were notable for a sodium of 131 , potassium 6.2 , BUN 111 , creatinine 3.9 , white blood cell count 23 , hematocrit 27 with 2+ schistocytes and 2+ spherocytes on the smear , platelet count 92 and LDH 496 . 
	 TEST on admission were notable for TEST of 131 , TEST 6.2 , TEST 111 , TEST 3.9 , TEST 23 , TEST 27 with PROBLEM and PROBLEM on TEST , TEST 92 and TEST 496 .


Predicted Relation: 	 TeRP, None 
Entities: 	 BUN, TEST 	 2+ spherocytes, PROBLEM 
Sentences: 
	Laboratory studies on admission were notable for a sodium of 131 , potassium 6.2 , BUN 111 , creatinine 3.9 , white blood cell count 23 , hematocrit 27 with 2+ schistocytes and 2+ spherocytes on the smear , platelet count 92 and LDH 496 . 
	 T